# Qubit spectroscopy: $T_1$ measurement example

This notebook demonstrates how to assess the decoherence time, $T_1$, of one or more qubits on a real quantum device using pyQuil. A $T_1$ experiment consists of an X pulse, bringing the qubit from $|0\rangle$ to $|1\rangle$ (the excited state of the artifical atom), followed by a delay of variable duration. 

The physics of the devices is such that We expect the state to decay exponentially with increasing time because of ["energy relaxation"](https://en.wikipedia.org/wiki/Relaxation_(NMR)). We characterize this decay by the time decay constant, which we call $\Gamma =1/T_1$. The parameter $T_1$ is refered to as the qubit's "relaxation" or "decoherence" time. A sample QUIL program at one data point (specified by the duration of the DELAY pragma) for qubit 0 with a 10us wait would look like 

    DECLARE ro BIT[1]
    RX(pi) 0
    PRAGMA DELAY 0 "1e-05"
    MEASURE 0 ro[0]

**NB: Since decoherence and dephasing noise are only simulated on gates, and we make use of DELAY pragmas to simulate relaxation time, we cannot simulate decoherence on the QPU with this experiment as written. This notebook should only be run on a real quantum device.**

## setup - imports and relevant units

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from pyquil.api import get_qc, QuantumComputer

from forest_benchmarking.qubit_spectroscopy import (
    generate_t1_experiments,
    acquire_data_t1,
    fit_t1,
    plot_t1_fit_over_data)

We treat SI base units, such as the second, as dimensionless, unit quantities, so we define relative units, such as the microsecond, using scientific notation. 

In [2]:
MICROSECOND = 1e-6

## measurement, analysis, and plotting

This method takes a `pandas.DataFrame` object with all the $T_1$ data, including the times and qubits measured and the resulting decay at each data point, and plots the decay of each qubit with respect to increasing delay duration. It also fits to an exponential decay curve, evaluates the fitted time decay constant, $T_1$, for each qubit, and plots the fitted curve on top of the experimental data so that a user can assess the fit if so desired. 

In [3]:
# qc = get_qc('Aspen-1-3Q-A')
qc = get_qc('2q-noisy-qvm') # will run on a QVM, but not meaningfully 
qubits = qc.qubits()


**Generate simultaneous $T_1$ experiments**

We can specify which qubits we want to measure using `qubits` and the maximum delay we'll use for each using `stop_time`. 

In [4]:
stop_time = 60 * MICROSECOND
num_shots = 1000
num_points = 15
qubits = [0,1,2]
t1_exp = generate_t1_experiments(qubits, stop_time, num_shots, num_points)

In [5]:
t1_exp

[(0.0, <pyquil.quil.Program at 0x1226caf60>),
 (4.3e-06, <pyquil.quil.Program at 0x1226cafd0>),
 (8.6e-06, <pyquil.quil.Program at 0x1226dd518>),
 (1.29e-05, <pyquil.quil.Program at 0x1226dda20>),
 (1.71e-05, <pyquil.quil.Program at 0x1226ddf28>),
 (2.14e-05, <pyquil.quil.Program at 0x1226e03c8>),
 (2.57e-05, <pyquil.quil.Program at 0x1226e0828>),
 (3e-05, <pyquil.quil.Program at 0x1226e0c88>),
 (3.43e-05, <pyquil.quil.Program at 0x1226e5128>),
 (3.86e-05, <pyquil.quil.Program at 0x1226e5630>),
 (4.29e-05, <pyquil.quil.Program at 0x1226e5a90>),
 (4.71e-05, <pyquil.quil.Program at 0x1226e5ef0>),
 (5.14e-05, <pyquil.quil.Program at 0x1226ea390>),
 (5.57e-05, <pyquil.quil.Program at 0x1226ea7f0>),
 (6e-05, <pyquil.quil.Program at 0x1226eac50>)]

**Acquire data**

Collect our $T_1$ raw data using `acquire_data_t1`. 

In [6]:
results = acquire_data_t1(qc,t1_exp)

**Use the results to produce estimates of $T_1$**

In [7]:
fit_t1(results)

Could not fit to experimental data for qubit 2


[{'qubit': 0,
  'T1': 212152.99741038436,
  'fit_params': array([ 9.10807892e-01,  2.12152997e-01, -1.89353178e-04]),
  'fit_params_errs': array([511.75475281,   6.16868706, 119.20113503]),
  'message': None},
 {'qubit': 1,
  'T1': 6677.964308942798,
  'fit_params': array([ 9.21027081e-01,  6.67796431e-03, -3.77528827e-05]),
  'fit_params_errs': array([4.49154135e+03, 8.13227571e-03, 3.25610051e+01]),
  'message': None},
 {'qubit': 2,
  'T1': None,
  'fit_params': None,
  'fit_params_errs': None,
  'message': 'Could not fit to experimental data for qubit2'}]

In [11]:
plot_t1_fit_over_data(results)

NameError: name 'plt' is not defined

Now we can use the above plotting method to measure $T_1$ on our lattice, `Aspen-1-3Q-A`. 

In [ ]:
plot_fitted_exponential_decay_curve_over_data(df, filename=f't1.{qc.name}.png')

In [4]:
q = 1
'Could not fit to experimental data for qubit' + str(q)

'Could not fit to experimental data for QC1'

In [9]:
df = pd.DataFrame({'qbit':[1,2,3]})

In [23]:
qbits = [1,3,4] 

for qbx in qbits:    
    if qbx not in df['qbit'].unique():
        raise ValueError("The list of qubits does not match the ones you experimented on.")
    print(qbx)


1
3


ValueError: The list of qubits does not match the ones you experimented on.

In [26]:
df['qbit'].unique().tolist()

[1, 2, 3]